In [4]:
import sys
sys.path.append('..')
from model import Model, UnetModel
import torch
from PIL import Image
import numpy as np
from process_data import (
    train_preprocess,
    val_preprocess,
    get_augmentations,
    get_data_loader,
)

In [5]:
# model = Model()
# model.load_state_dict(torch.load("../model_0.27.pth"))

class Args:
    def __init__(self):
        self.batch_size = 1
        self.num_workers = 1
        self.data_path = "../data/CityScapes"
        
args = Args()

In [6]:
# Get dataloaders
train_loader, val_loader = get_data_loader(
    args=args, batch_size=2, num_workers=1, validation_size=0.1
)

In [7]:
# Sample a batch of images
train_batch = next(iter(train_loader))
val_batch = next(iter(val_loader))

In [8]:
# Check properties of both train and val batch
print("Train batch:")
print(train_batch[0].shape)
print(train_batch[1].shape)
print(train_batch[0].dtype)
print(train_batch[1].dtype)

print("Val batch:")
print(val_batch[0].shape)
print(val_batch[1].shape)
print(val_batch[0].dtype)
print(val_batch[1].dtype)

# Also get min max values of the images
print("Train batch min max:")
print(train_batch[0].min())
print(train_batch[0].max())

print("Val batch min max:")
print(val_batch[0].min())
print(val_batch[0].max())


Train batch:
torch.Size([2, 3, 520, 1040])
torch.Size([2, 1, 520, 1040])
torch.float32
torch.uint8
Val batch:
torch.Size([2, 3, 520, 1040])
torch.Size([2, 1, 520, 1040])
torch.float32
torch.uint8
Train batch min max:
tensor(-2.1179)
tensor(2.6400)
Val batch min max:
tensor(-1.9980)
tensor(2.5877)


In [9]:
model = UnetModel()
model.load_state_dict(torch.load("../model_0296.pth"))



<All keys matched successfully>

In [14]:
print(val_batch[0][0].shape)

torch.Size([3, 520, 1040])


In [17]:
# Make a prediction with the model
model.eval()
with torch.no_grad():
    prediction = model(val_batch)

# Convert prediction to image
prediction = prediction[0].cpu().numpy()
prediction = np.argmax(prediction, axis=0)

# Convert to image
prediction = Image.fromarray(prediction.astype(np.uint8))
prediction.show()

# Show original image
original = val_batch[0][0].cpu().numpy()
original = np.transpose(original, (1, 2, 0))
original = Image.fromarray(original.astype(np.uint8))

original.show()

ValueError: expected 4D input (got 3D input)